In [1]:
import json

import pandas as pd
import numpy as np

import networkx as nx
from graph_encoders import *

from linear_algebra_utils import *
from digraph_algorithms import *
from quiver_pca import *

In [2]:
X_train = pd.read_csv('../data/crop_mapping/node_reduced_train.csv')
X_test = pd.read_csv('../data/crop_mapping/node_reduced_test.csv')

with open('../data/crop_mapping/quiver_representation.json', 'r') as file:
    Q = json.load(file, cls=NetworkXQuiverRepresentationDecoder)

with open('../data/crop_mapping/reduced_nodes.json', 'r') as file:
    Q_nodes = json.load(file)

In [3]:
# Augment quiver with root vertex and generate space of sections
Qplus = augment_DAG_with_root(Q)
generate_space_of_sections(Qplus)

In [4]:
# Generate orthonormal basis for the space of sections
sections = Qplus.graph['sections']
normalized_sections = orthogonalize_section_basis(sections, Q_nodes)

In [5]:
# Project onto orthonormal basis of sections, in the section basis' coordinates
X_train_proj = [project_onto_subspace(X_train.iloc[i], normalized_sections) for i in range(len(X_train))]
X_train_proj = pd.concat(X_train_proj, axis=1).T
X_test_proj = [project_onto_subspace(X_test.iloc[i], normalized_sections) for i in range(len(X_test))]
X_test_proj = pd.concat(X_test_proj, axis=1).T

In [6]:
Q_cols = ['Q1_' + str(i) for i in range(len(normalized_sections))]
X_train_proj.columns = Q_cols
X_test_proj.columns = Q_cols

In [7]:
X_train_proj.head()

,Q1_0,Q1_1,Q1_2,Q1_3,Q1_4,Q1_5,Q1_6,Q1_7,Q1_8
0,3.986259,-2.237170,-6.853211,6.399514,2.610451,-5.483664,-2.580249,-1.310271,-3.664420
1,2.505083,-1.512982,-6.095646,4.446712,2.930806,-4.792045,-0.931142,-2.199914,-3.848712
2,2.711012,-2.366670,-5.482358,4.512390,2.845891,-4.484513,-1.001577,-1.853035,-3.140195
3,2.551358,-0.886571,-6.754687,4.604886,3.054138,-5.490920,-1.208640,-2.650506,-4.509557
4,2.802179,-1.278422,-4.658371,4.025560,2.494252,-3.935713,-0.944215,-1.478768,-2.480393


In [8]:
X_train_proj.to_csv('../data/crop_mapping/quiver_invariant_train.csv', mode='w', index=False)
X_test_proj.to_csv('../data/crop_mapping/quiver_invariant_test.csv', mode='w', index=False)